In [4]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import operator
import matplotlib.cm as cm
import matplotlib.patches as patches
import math
import time
import json
import requests
import hide_code
import re
from sklearn import cross_validation
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import GradientBoostingClassifier as GradientBoosting

#Things to try: Logistic Regression, Random Forest, Gradient Boosting

# help(hide_code)
import notebook
# E = notebook.nbextensions.EnableNBExtensionApp()

# E.enable_nbextension('usability/codefolding/main')

# from IPython.display import HTML
# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# <form action="javascript:code_toggle()"><input type="submit" value="low-key"></form>''')





In [5]:
train = pd.read_csv('cs-training.csv')

In [6]:
train.describe(include='all').round(2)
# Get rid of extra index-like variables (Unamed: 0)
del train[train.columns[0]]
train.describe(include='all').round(2)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.00,150000.00,150000.00,150000.00,150000.00,120269.00,150000.00,150000.00,150000.00,150000.00,146076.00
mean,0.07,6.05,52.30,0.42,353.01,6670.22,8.45,0.27,1.02,0.24,0.76
std,0.25,249.76,14.77,4.19,2037.82,14384.67,5.15,4.17,1.13,4.16,1.12
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.03,41.00,0.00,0.18,3400.00,5.00,0.00,0.00,0.00,0.00
50%,0.00,0.15,52.00,0.00,0.37,5400.00,8.00,0.00,1.00,0.00,0.00
75%,0.00,0.56,63.00,0.00,0.87,8249.00,11.00,0.00,2.00,0.00,1.00
max,1.00,50708.00,109.00,98.00,329664.00,3008750.00,58.00,98.00,54.00,98.00,20.00


In [7]:
# Determine which features require preprocessing.
def list_features_wmissing(dataset):
    '''
    Returns all features that have missing values:
        - a list of just those features.'''
    print('Summary Statistics on Full Data set:\n{}'.format(dataset.describe(include='all').round(2)))
    has_null = pd.DataFrame({'Total_missings' : dataset.isnull().sum()})
    has_null[(has_null.Total_missings >0)].index.tolist()
    print('\n\n{} Features containing missing values: {}\n'
          .format(len(has_null[(has_null.Total_missings >0)].index.tolist()),
           has_null[(has_null.Total_missings >0)].index.tolist()))
    print(has_null.ix[2:,:])
    return has_null[(has_null.Total_missings >0)].index.tolist()

feature_wnull = list_features_wmissing(train)

Summary Statistics on Full Data set:
       SeriousDlqin2yrs  RevolvingUtilizationOfUnsecuredLines        age  \
count         150000.00                             150000.00  150000.00   
mean               0.07                                  6.05      52.30   
std                0.25                                249.76      14.77   
min                0.00                                  0.00       0.00   
25%                0.00                                  0.03      41.00   
50%                0.00                                  0.15      52.00   
75%                0.00                                  0.56      63.00   
max                1.00                              50708.00     109.00   

       NumberOfTime30-59DaysPastDueNotWorse  DebtRatio  MonthlyIncome  \
count                             150000.00  150000.00      120269.00   
mean                                   0.42     353.01        6670.22   
std                                    4.19    2037.82     

In [8]:
# Feed this list as input to a general function that 
# trains a model of missings imputations for each
def get_correlates_dict(dataset, feature_wnull, not_same=True):
    correlated_predictionary = {}
    for target in feature_wnull:
        try:
            corrMatrix = pd.DataFrame({'Correlation': dataset.corr().ix[:,target].sort_values()})
            corrAbsMatrix = corrMatrix.copy()
            corrAbsMatrix['Absolute'] = corrAbsMatrix.Correlation.apply(lambda x : abs(x)) 
            corrAbsMatrix.sort_values('Absolute', inplace=True)
            print('\nCorrelation Matrix with Respect to {}:\n'.format(target),
                  corrAbsMatrix)
            iteration = -3
            best_correlates = corrAbsMatrix.index.tolist()\
                              [iteration : len(corrAbsMatrix.index.tolist())]
            if not_same:
                # Remove any similarly named correlates that 
                # likely to be transformations of the same target variable.
                exclude_by_name = []
                for c in corrAbsMatrix.index.tolist():
                    similar = re.search(target, c)
                    if similar:
                        exclude_by_name.append(c)
                best_correlates = [choice for choice in best_correlates 
                                   if choice not in exclude_by_name]
                iteration = -3
                while len(best_correlates) < 3:
                    iteration -= 1
                    best_correlates = corrAbsMatrix.index.tolist()\
                                      [iteration : len(corrAbsMatrix.index.tolist())]
                    best_correlates = [choice for choice in best_correlates 
                                       if choice not in exclude_by_name]  
            print('\nTOP 3 Correlates with {}: (EXCLUDES {})\n{}\n\n{}'.
                  format(target, exclude_by_name, corrAbsMatrix.tail(abs(iteration)), best_correlates))
            correlated_predictionary[target] = best_correlates
        except:
            print('get_correlates_dict ERROR: {}'.format(target))
    return correlated_predictionary

correlated_features = get_correlates_dict(train, feature_wnull)


Correlation Matrix with Respect to MonthlyIncome:
                                       Correlation  Absolute
RevolvingUtilizationOfUnsecuredLines     0.007124  0.007124
NumberOfTime30-59DaysPastDueNotWorse    -0.010217  0.010217
NumberOfTime60-89DaysPastDueNotWorse    -0.011116  0.011116
NumberOfTimes90DaysLate                 -0.012743  0.012743
SeriousDlqin2yrs                        -0.019746  0.019746
DebtRatio                               -0.028712  0.028712
age                                      0.037717  0.037717
NumberOfDependents                       0.062647  0.062647
NumberOfOpenCreditLinesAndLoans          0.091455  0.091455
NumberRealEstateLoansOrLines             0.124959  0.124959
MonthlyIncome                            1.000000  1.000000

TOP 3 Correlates with MonthlyIncome: (EXCLUDES ['MonthlyIncome'])
                                 Correlation  Absolute
NumberOfDependents                  0.062647  0.062647
NumberOfOpenCreditLinesAndLoans     0.091455  0.091

In [9]:
# Feed this dictionary into a function that 
# trains a model of missings imputations
def get_encoded_features_list(dataset, discrete_threshold=None, excepting=None, only=None):
    '''Returns a list of discrete variables that
       have numerical values signifying missing.
       (i.e. 96=DK, 98=REFUSED)'''
    contain_missing = []
    print('\nINSPECT DISCRETE FEATURES FOR ENCODED MISSISINGS:\n')
    if not only:
        only = dataset.columns.tolist()
    only = [feature for feature in only if feature not in excepting]
    for col in only:
        if discrete_threshold:
            if len(dataset[col].unique()) < discrete_threshold:
                print('{} unique values:\n{}'.format(col, dataset[col].unique()))
                contain_missing += [col]
        else:
            print('{} unique values:\n{}'.format(col, dataset[col].unique()))
            contain_missing += [col]
    print('\nCheck for coded missings on these features: \n{}'.format(contain_missing))
    return contain_missing


def decode_extended_to_nan(dataset, contain_missing, to_replace=None, values=None):
    '''Inputs: Dataframe and list of features to encode.
       Returns a dataframe with specified encoded missings
       replaced to NAN.'''
    if (len(to_replace) == len(values)) & (isinstance(to_replace, list)):
        dataset[contain_missing] = dataset[contain_missing].replace(to_replace, values)
        new_missings = pd.DataFrame({'Total_missing' : 
                                    dataset.isnull().sum().round(2)})
        print('\nEncode obs with NAN in any of the extended missings columns: \n{}\
              \nAfter Encoding Extended Missings\n{},\n\nDrop obs with extended missings'.
              format(contain_missing, new_missings))
        return dataset, new_missings
    else:
        raise ValueError("Provide symmetrical number lists to 'to_replace' and 'values'.")
        
        
def drop_obs_w_anynan(dataset, features_list):
    '''Inputs: Dataframe and list of features to encode.
       Returns a dataset removed any obs missing
       on the specified by the features_list.'''
    excludes = [feature for feature in dataset.columns.tolist() if feature not in features_list]
    if excludes:
        print('\nDropping all row-wise missing values, EXCEPT for features:\n{}'
             .format(excludes))
    else:
        print('\nDropping all row-wise missing values for features:\n{}'
              .format(features_list))
    dataset.dropna(axis='index', how='any', inplace=True, subset=[features_list])
    print('\nAfter Dropping Extended Missings\n{}\n'
          .format(pd.DataFrame(
                 {'Total_missings' : dataset.isnull().sum()})))
    return dataset


def decode_and_drop_missings(raw_train, decodings_dict, except_threshold=None, encode_except=None):
    '''Inputs:
        - Dataframe 
        - Decoding variables to values dictionary
        - Optional discrete_threshold (i.e. 1000 unique values)
       Returns '''
    encoded_features = get_encoded_features_list(raw_train, 
                                                 except_threshold, 
                                                 excepting=encode_except)
    for i in range(len(decodings_dict.keys())):
        if not decodings_dict[i]['on']:
            decodings_dict[i]['on'] = encoded_features
        else:
            encoded_features.extend(decodings_dict[i]['on'])
        raw_train, imputation_candidates = \
        decode_extended_to_nan(raw_train, decodings_dict[i]['on'], 
                               to_replace = decodings_dict[i]['to_replace'], 
                               values = decodings_dict[i]['with_replace'])    
    # Derive binary missing indicator variables
    dropped_train = raw_train.copy()
    derived_train = raw_train.copy()
    for feature in imputation_candidates[imputation_candidates.Total_missing > 0].index:
        is_missing_var = feature + '_missing'
        derived_train[is_missing_var] = derived_train[feature].isnull().map({True : 1, False : 0})
        print('\nDERIVED BINARY MISSING VARIABLES {}\n'.format(is_missing_var))
        print(derived_train[[feature, is_missing_var]].describe())
    # Drop all missings 
    derived_train.dropna(how='any', axis=1)
    dropped_train =  drop_obs_w_anynan(dropped_train, encoded_features).copy()
    return dropped_train, derived_train, imputation_candidates[imputation_candidates.Total_missing > 0].index.tolist() 

In [10]:
decodings_dict = {0 :{'on': None, 'to_replace' : [96, 98], 
                  'with_replace' : [np.nan, np.nan]},
                  1:{'on': ['age'],
                  'to_replace' : [0], 
                  'with_replace' : [np.nan]}}

train_alldropped, derived_train, imputation_candidates = \
decode_and_drop_missings(train, decodings_dict, except_threshold=None, encode_except=['age','Unnamed: 0'])


INSPECT DISCRETE FEATURES FOR ENCODED MISSISINGS:

SeriousDlqin2yrs unique values:
[1 0]
RevolvingUtilizationOfUnsecuredLines unique values:
[ 0.76612661  0.95715102  0.65818014 ...,  0.29974515  0.24604392
  0.85028295]
NumberOfTime30-59DaysPastDueNotWorse unique values:
[ 2  0  1  3  4  5  7 10  6 98 12  8  9 96 13 11]
DebtRatio unique values:
[ 0.80298213  0.1218762   0.08511338 ...,  0.40429286  0.71656222
  0.24990808]
MonthlyIncome unique values:
[  9120.   2600.   3042. ...,  15648.    521.   7647.]
NumberOfOpenCreditLinesAndLoans unique values:
[13  4  2  5  7  3  8  9  6 10 14 22 17  1 15 31 12 21 11 19 18 26 20 23  0
 16 27 29 28 24 25 32 46 33 30 34 37 38 36 35 54 43 45 49 57 48 40 58 51 52
 39 41 56 42 47 50 44 53]
NumberOfTimes90DaysLate unique values:
[ 0  1  3  2  5  4 98 10  9  6  7  8 15 96 11 13 14 17 12]
NumberRealEstateLoansOrLines unique values:
[ 6  0  1  3  4  2  5  8  7  9 17 15 10 12 11 16 26 14 54 21 18 13 23 29 25
 19 20 32]
NumberOfTime60-89DaysPastDueNotWo

In [11]:
derived_train.dropna(how='any', axis=1)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,NumberOfOpenCreditLinesAndLoans,NumberRealEstateLoansOrLines,age_missing,NumberOfTime30-59DaysPastDueNotWorse_missing,DebtRatio_missing,MonthlyIncome_missing,NumberOfTimes90DaysLate_missing,NumberOfTime60-89DaysPastDueNotWorse_missing,NumberOfDependents_missing
0,1,0.766127,13,6,0,0,0,0,0,0,0
1,0,0.957151,4,0,0,0,0,0,0,0,0
2,0,0.658180,2,0,0,0,0,0,0,0,0
3,0,0.233810,5,0,0,0,0,0,0,0,0
4,0,0.907239,7,1,0,0,0,0,0,0,0
5,0,0.213179,3,1,0,0,0,0,0,0,0
6,0,0.305682,8,3,0,0,0,1,0,0,0
7,0,0.754464,8,0,0,0,0,0,0,0,0
8,0,0.116951,2,0,0,0,0,1,0,0,1
9,0,0.189169,9,4,0,0,0,0,0,0,0


In [12]:
derived_train.columns, imputation_candidates

(Index(['SeriousDlqin2yrs', 'RevolvingUtilizationOfUnsecuredLines', 'age',
        'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
        'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
        'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
        'NumberOfDependents', 'age_missing',
        'NumberOfTime30-59DaysPastDueNotWorse_missing', 'DebtRatio_missing',
        'MonthlyIncome_missing', 'NumberOfTimes90DaysLate_missing',
        'NumberOfTime60-89DaysPastDueNotWorse_missing',
        'NumberOfDependents_missing'],
       dtype='object'),
 ['age',
  'NumberOfTime30-59DaysPastDueNotWorse',
  'DebtRatio',
  'MonthlyIncome',
  'NumberOfTimes90DaysLate',
  'NumberOfTime60-89DaysPastDueNotWorse',
  'NumberOfDependents'])

In [13]:
print('\nSummarized Data After Removing Cases with Missing Values:\n')
train_alldropped.describe(include='all').round(2)


Summarized Data After Removing Cases with Missing Values:



,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,120115.00,120115.00,120115.00,120115.00,120115.00,120115.00,120115.00,120115.00,120115.00,120115.00,120115.00
mean,0.07,5.91,51.31,0.26,26.63,6675.57,8.77,0.09,1.06,0.07,0.85
std,0.25,257.21,14.42,0.72,424.72,14392.78,5.17,0.49,1.15,0.34,1.15
min,0.00,0.00,21.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.00,0.03,41.00,0.00,0.14,3400.00,5.00,0.00,0.00,0.00,0.00
50%,0.00,0.18,51.00,0.00,0.30,5400.00,8.00,0.00,1.00,0.00,0.00
75%,0.00,0.58,61.00,0.00,0.48,8250.00,11.00,0.00,2.00,0.00,2.00
max,1.00,50708.00,103.00,13.00,61106.50,3008750.00,58.00,17.00,54.00,11.00,20.00


In [14]:
# Transform Data

def gen_transform_data(dataset, transform_dict, transformations=None):
    '''Inputs: 
        - Dataset
        - dictionary of features to transform with
            respective transformation function.'''
    if not transformations:            
        transformations = {'log': 'np.log'}
    for feature in transform_dict.keys():
        fx = transform_dict[feature]
        plus = 0
        if fx in transformations:
            new_feature = feature + '_' + fx
            print('\nTransforming {} by way of {} = {}'
                  .format(feature, fx, new_feature))
            if fx == 'log':
                plus = 1
            dataset[new_feature] = dataset[feature].apply(lambda x : eval(transformations[fx])(x + plus))
            print(dataset[[feature, new_feature]].head())
        else:
            raise ValueError("Provide valid transformation function. {} is invalid.".format(fx))
    return dataset

transform_features_dict = {'RevolvingUtilizationOfUnsecuredLines' : 'log', 
                           'MonthlyIncome' : 'log'}

train_transformed = gen_transform_data(train_alldropped, transform_features_dict)
train_derived_transformed = gen_transform_data(derived_train, transform_features_dict)


Transforming MonthlyIncome by way of log = MonthlyIncome_log
   MonthlyIncome  MonthlyIncome_log
0         9120.0           9.118335
1         2600.0           7.863651
2         3042.0           8.020599
3         3300.0           8.101981
4        63588.0          11.060196

Transforming RevolvingUtilizationOfUnsecuredLines by way of log = RevolvingUtilizationOfUnsecuredLines_log
   RevolvingUtilizationOfUnsecuredLines  \
0                              0.766127   
1                              0.957151   
2                              0.658180   
3                              0.233810   
4                              0.907239   

   RevolvingUtilizationOfUnsecuredLines_log  
0                                  0.568789  
1                                  0.671490  
2                                  0.505721  
3                                  0.210107  
4                                  0.645657  

Transforming MonthlyIncome by way of log = MonthlyIncome_log
   MonthlyIncome 

In [15]:
train_transformed.columns

Index(['SeriousDlqin2yrs', 'RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents', 'MonthlyIncome_log',
       'RevolvingUtilizationOfUnsecuredLines_log'],
      dtype='object')

In [16]:
dropped_correlates_dict = get_correlates_dict(train_transformed, train_transformed.columns.tolist(), not_same=True)


Correlation Matrix with Respect to SeriousDlqin2yrs:
                                           Correlation  Absolute
NumberRealEstateLoansOrLines                -0.000803  0.000803
RevolvingUtilizationOfUnsecuredLines        -0.002351  0.002351
DebtRatio                                   -0.003153  0.003153
MonthlyIncome_log                           -0.016464  0.016464
MonthlyIncome                               -0.019124  0.019124
NumberOfOpenCreditLinesAndLoans             -0.023514  0.023514
NumberOfDependents                           0.047311  0.047311
age                                         -0.101074  0.101074
RevolvingUtilizationOfUnsecuredLines_log     0.175496  0.175496
NumberOfTime60-89DaysPastDueNotWorse         0.261824  0.261824
NumberOfTime30-59DaysPastDueNotWorse         0.273399  0.273399
NumberOfTimes90DaysLate                      0.306880  0.306880
SeriousDlqin2yrs                             1.000000  1.000000

TOP 3 Correlates with SeriousDlqin2yrs: (EXCLUDE

In [17]:
# Recall imputation candidates (features with missing data before drop)
print('CONSIDER THESE IMPUTATION CANDIDATES AFTER LOOKING AT COEFFICIENTS OF FIRST APPROXIMATION:\n',  imputation_candidates)

# Recall train_derived_transformed (contains missings binaries)
train_derived_transformed.isnull().sum()

# Keep a training set version which uses the binary versions for any variables that have missings
train_derived_transformed_dropped = train_derived_transformed.copy()
for feature in imputation_candidates:
    del train_derived_transformed_dropped[feature]
    try:
        del train_derived_transformed_dropped[feature + '_log']
    except:
        print('ERROR:{}'.format(feature + '_log'))
print('\nSummarize Training Set Version that Replaces Imputation Candidates with Binary Missings Variables:\n')
train_derived_transformed_dropped
train_derived_transformed_dropped.isnull().sum()

CONSIDER THESE IMPUTATION CANDIDATES AFTER LOOKING AT COEFFICIENTS OF FIRST APPROXIMATION:
 ['age', 'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome', 'NumberOfTimes90DaysLate', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']
ERROR:age_log
ERROR:NumberOfTime30-59DaysPastDueNotWorse_log
ERROR:DebtRatio_log
ERROR:NumberOfTimes90DaysLate_log
ERROR:NumberOfTime60-89DaysPastDueNotWorse_log
ERROR:NumberOfDependents_log

Summarize Training Set Version that Replaces Imputation Candidates with Binary Missings Variables:



SeriousDlqin2yrs                                0
RevolvingUtilizationOfUnsecuredLines            0
NumberOfOpenCreditLinesAndLoans                 0
NumberRealEstateLoansOrLines                    0
age_missing                                     0
NumberOfTime30-59DaysPastDueNotWorse_missing    0
DebtRatio_missing                               0
MonthlyIncome_missing                           0
NumberOfTimes90DaysLate_missing                 0
NumberOfTime60-89DaysPastDueNotWorse_missing    0
NumberOfDependents_missing                      0
RevolvingUtilizationOfUnsecuredLines_log        0
dtype: int64

In [18]:
ismiss_correlates_dict = get_correlates_dict(train_derived_transformed_dropped, train_transformed.columns.tolist(), not_same=True)


Correlation Matrix with Respect to SeriousDlqin2yrs:
                                               Correlation  Absolute
age_missing                                     -0.000691  0.000691
RevolvingUtilizationOfUnsecuredLines            -0.001802  0.001802
DebtRatio_missing                                0.002674  0.002674
NumberRealEstateLoansOrLines                    -0.007038  0.007038
NumberOfDependents_missing                      -0.013928  0.013928
MonthlyIncome_missing                           -0.021308  0.021308
NumberOfOpenCreditLinesAndLoans                 -0.029669  0.029669
NumberOfTime30-59DaysPastDueNotWorse_missing     0.081401  0.081401
NumberOfTimes90DaysLate_missing                  0.081401  0.081401
NumberOfTime60-89DaysPastDueNotWorse_missing     0.081401  0.081401
RevolvingUtilizationOfUnsecuredLines_log         0.178767  0.178767
SeriousDlqin2yrs                                 1.000000  1.000000

TOP 3 Correlates with SeriousDlqin2yrs: (EXCLUDES ['SeriousD

In [19]:
# Review variables that are most correlated in both versions of the transformed data.
print("Review variable correlations for each version of the \
'NON-MISSING' transformed training data:\
\n-Casewise drop of any examples with missings:")
dropped_correlates_dict     

Review variable correlations for each version of the 'NON-MISSING' transformed training data:
-Casewise drop of any examples with missings:


{'DebtRatio': ['NumberRealEstateLoansOrLines',
  'MonthlyIncome',
  'MonthlyIncome_log'],
 'MonthlyIncome': ['NumberOfDependents',
  'NumberOfOpenCreditLinesAndLoans',
  'NumberRealEstateLoansOrLines'],
 'MonthlyIncome_log': ['NumberRealEstateLoansOrLines',
  'MonthlyIncome',
  'DebtRatio'],
 'NumberOfDependents': ['MonthlyIncome_log',
  'NumberRealEstateLoansOrLines',
  'age'],
 'NumberOfOpenCreditLinesAndLoans': ['MonthlyIncome_log',
  'age',
  'NumberRealEstateLoansOrLines'],
 'NumberOfTime30-59DaysPastDueNotWorse': ['NumberOfTimes90DaysLate',
  'SeriousDlqin2yrs',
  'NumberOfTime60-89DaysPastDueNotWorse'],
 'NumberOfTime60-89DaysPastDueNotWorse': ['SeriousDlqin2yrs',
  'NumberOfTimes90DaysLate',
  'NumberOfTime30-59DaysPastDueNotWorse'],
 'NumberOfTimes90DaysLate': ['NumberOfTime30-59DaysPastDueNotWorse',
  'NumberOfTime60-89DaysPastDueNotWorse',
  'SeriousDlqin2yrs'],
 'NumberRealEstateLoansOrLines': ['MonthlyIncome',
  'MonthlyIncome_log',
  'NumberOfOpenCreditLinesAndLoans'],
 '

In [20]:
print("\n-Column wise drop of columns with missing examples, \
replaced by '_missing' binary variables:")
ismiss_correlates_dict


-Column wise drop of columns with missing examples, replaced by '_missing' binary variables:


{'NumberOfOpenCreditLinesAndLoans': ['RevolvingUtilizationOfUnsecuredLines_log',
  'MonthlyIncome_missing',
  'NumberRealEstateLoansOrLines'],
 'NumberRealEstateLoansOrLines': ['NumberOfDependents_missing',
  'MonthlyIncome_missing',
  'NumberOfOpenCreditLinesAndLoans'],
 'RevolvingUtilizationOfUnsecuredLines': ['NumberOfDependents_missing',
  'NumberRealEstateLoansOrLines',
  'NumberOfOpenCreditLinesAndLoans'],
 'RevolvingUtilizationOfUnsecuredLines_log': ['NumberOfOpenCreditLinesAndLoans',
  'SeriousDlqin2yrs',
  'RevolvingUtilizationOfUnsecuredLines'],
 'SeriousDlqin2yrs': ['NumberOfTimes90DaysLate_missing',
  'NumberOfTime60-89DaysPastDueNotWorse_missing',
  'RevolvingUtilizationOfUnsecuredLines_log']}

In [21]:
imputation_candidates

['age',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'DebtRatio',
 'MonthlyIncome',
 'NumberOfTimes90DaysLate',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'NumberOfDependents']

In [22]:
# Build Model and Results Dictionaries:
# Try predicting on the target outcome variable, using each version
# of the training set. These serve as the most agnostic to feature importance. 
# Try predicting each imputation candidate using each version.

model_builder = {'trainees': [train_transformed, train_derived_transformed_dropped], 
                 'test_sizes': [.20, .20],
                 'trainers': {'RandomForest':{'n_estimators': [10, 100, 1000],
                                              'min_samples_split': [1, 20, 50, 100]},
                              'DecisionTree':{'max_depth': [None, 10, 100, 1000], 
                                              'min_samples_split':[1, 20, 50, 100]}
                              #,'GradientBoosting':{'n_estimators': [10, 100, 1000],
                                                  #'min_samples_split': [1, 20, 50, 100]}
                             },
                 'outcome': 'SeriousDlqin2yrs'
                 }


def get_mse(predicted, val_targets):
    return (((predicted - val_targets) ** 2).sum()) / len(predicted)
                     

In [ ]:
def split_and_run(classifier_dictionary, 
                  cols=None, test_size=0.20, 
                  results_dataframe=None, 
                  filename='output'):
    # X_train, X_val = cross_validation.train_test_split(train_transformed, test_size = 0.20)
    results_matrix = {'Model': [], 
                      'Training_set': [], 
                      'Y_outcome' : [],
                      'Test_size': [],
                      'Classifier': [],
                      'max_depth': [],
                      'n_estimators': [],
                      'min_samples_split': [],
                      'Predictors': [],
                      'score': [],
                      'metric': []
                     }
    write_to = filename + '.xlsx'
    outcome_variable = classifier_dictionary['outcome']
    # Loop through models in classifier_dictionary
    model_number = 0
    if not results_dataframe:
        results_dataframe = results_matrix.copy()
    
    
    for dataset in range(len(classifier_dictionary['trainees'])):
        # Split the training data into a training set and a validation set
        if not cols:
            cols = classifier_dictionary['trainees'][dataset].columns.tolist()
            cols = [f for f in cols if f not in [outcome_variable]]
            print('X Features:')
            print(cols)
        for testsize in classifier_dictionary['test_sizes']:
            X_train, X_val = cross_validation.train_test_split(classifier_dictionary['trainees'][dataset].copy(),
                                                               test_size = testsize)
            
            # Build the Random Forest Classifier
            for key in classifier_dictionary['trainers']:
                if key=='DecisionTree':
                    argument = 'max_depth'
                    counter_arg = 'n_estimators'
                else:
                    argument = 'n_estimators'
                    counter_arg = 'max_depth'
                    for estimator_num in classifier_dictionary['trainers'][key][argument]:                            
                        for min_sample_split in classifier_dictionary['trainers'][key]['min_samples_split']: 
                            
                            model_number += 1
                        
                            results_dataframe['Model'].append(model_number)
                            results_dataframe['Y_outcome'].append(outcome_variable)
                            results_dataframe['Training_set'].append(dataset)
                            results_dataframe['Test_size'].append(testsize)
                            results_dataframe['Classifier'].append(key)
                            results_dataframe[argument].append(estimator_num)
                            results_dataframe[counter_arg].append(np.nan)
                            
                            # Set model
                            if argument == 'n_estimators':
                                clf = eval(key)(n_estimators = estimator_num, min_samples_split=min_sample_split)
                            elif argument == 'max_depth':
                                clf = eval(key)(max_depth = estimator_num, min_samples_split=min_sample_split)

                            #Fit the model to the training inputs and training targets
                            results_dataframe['Predictors'].append(cols)
                            clf.fit(X_train[cols].as_matrix(), X_train[outcome_variable].as_matrix())

                            #Predict the the output on the validation
                            predicted = clf.predict(X_val[cols].as_matrix())
                            print('Model {}.'.format(model_number))
                            print('Training_set {}.'.format(dataset))
                            print('Sum of validation Y:',X_val[outcome_variable].sum())
                            print('Sum of predicted Y:', predicted.sum())
                            print('Size of Training Set:', X_train.size, ', Size of Validation Set:',X_val.size)
                            X_val[outcome_variable+'_predicted'] = predicted
                            mse = get_mse(predicted, X_val[outcome_variable].as_matrix())
                            results_dataframe['score'].append(mse)
                            results_dataframe['metric'].append('mse')
                            del X_val[outcome_variable+'_predicted']
                            print('Model metric: {}, Score: {}'.format('mse', mse))
                            results = pd.DataFrame.from_dict(results_dataframe)
                            results.to_excel(write_to)
    return pd.DataFrame.from_dict(results_dataframe).copy()


In [ ]:
results_df = split_and_run(model_builder)

Model 1.
Training_set 0.
Sum of validation Y: 1659
Sum of predicted Y: 512
Size of Training Set: 1249196 , Size of Validation Set: 312299
Model metric: mse, Score: 0.06930857927819173
Model 2.
Training_set 0.
Sum of validation Y: 1659
Sum of predicted Y: 544
Size of Training Set: 1249196 , Size of Validation Set: 336322
Model metric: mse, Score: 0.06631145152562128
Model 3.
Training_set 0.
Sum of validation Y: 1659
Sum of predicted Y: 467
Size of Training Set: 1249196 , Size of Validation Set: 336322
Model metric: mse, Score: 0.06593681055654997
Model 4.
Training_set 0.
Sum of validation Y: 1659
Sum of predicted Y: 372
Size of Training Set: 1249196 , Size of Validation Set: 336322
Model metric: mse, Score: 0.06522915539274862
Model 5.
Training_set 0.
Sum of validation Y: 1659
Sum of predicted Y: 517
Size of Training Set: 1249196 , Size of Validation Set: 336322
Model metric: mse, Score: 0.06602006410523248
Model 6.
Training_set 0.
Sum of validation Y: 1659
Sum of predicted Y: 489
Size 

In [ ]:
# results_df = split_and_run(model_builder)
# results_df.head()

In [ ]:

# #Build the Random Forest Classifier
# clf = RandomForest(n_estimators = 100, min_samples_split=50)

# #Fit the model to the training inputs and training targets
# clf.fit(X_train[cols],Y_train)

# #Predict the the output on the validation
# pred = clf.predict(X_val[cols])
# print(X_val['SeriousDlqin2yrs'].sum())
# print(pred.sum())

# #Still need a metric

In [ ]:
# data_dictionary = pd.read_csv('Data_Dictionary.csv')
# # print(data_dictionary)
# data_dictionary.columns #
# data_dictionary['Unnamed: 1']

In [ ]:
# results_df.head()